# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [71]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [72]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/jovyan/work


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [73]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [74]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [75]:
from cassandra.cluster import Cluster

try: 
    # we are using a cassandra database on a docker container called cassandra
    cluster = Cluster(['cassandra']) 
    
    # create a session to establish a connection and for executing queries on
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [76]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project_music_db 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")

except Exception as e:
    print(e)

#### Set Keyspace

In [77]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('project_music_db')
except Exception as e:
    print(e)

In [78]:
# Load the data into a pandas dataframe

data = pd.read_csv('event_datafile_new.csv')
data.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,matchbox twenty,Jayden,F,0,Duffy,177.65832,free,"Seattle-Tacoma-Bellevue, WA",846,Argue (LP Version),76
1,The Lonely Island / T-Pain,Jayden,F,1,Duffy,156.23791,free,"Seattle-Tacoma-Bellevue, WA",846,I'm On A Boat,76
2,John Mayer,Wyatt,M,0,Scott,275.27791,free,"Eureka-Arcata-Fortuna, CA",856,All We Ever Do Is Say Goodbye,9
3,10_000 Maniacs,Wyatt,M,2,Scott,251.87220,free,"Eureka-Arcata-Fortuna, CA",856,Gun Shy (LP Version),9
4,Leona Lewis,Chloe,F,0,Cuevas,203.88526,paid,"San Francisco-Oakland-Hayward, CA",916,Forgive Me,49


In [79]:
data.shape

(6820, 11)

In [80]:
columns = data.columns.tolist()
print(columns)

['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'sessionId', 'song', 'userId']


In [81]:
# check if there are missing values
data.isnull().sum()

artist           0
firstName        0
gender           0
itemInSession    0
lastName         0
length           0
level            0
location         0
sessionId        0
song             0
userId           0
dtype: int64

In [82]:
# get data info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


In [83]:
# check columns data types
data.dtypes

artist            object
firstName         object
gender            object
itemInSession      int64
lastName          object
length           float64
level             object
location          object
sessionId          int64
song              object
userId             int64
dtype: object

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

***

---

### Question 1
> Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

---

In [84]:
# this query creates a table for artist, song title and song's length in the music app history that was heard during a 
# particular session and for a specific itemInSession number.
query = """
        CREATE TABLE  IF NOT EXISTS artist_song_sessions (
            sessionId INT,
            itemInSession INT,
            artist TEXT,
            song TEXT, 
            length FLOAT,
            PRIMARY KEY (sessionId, itemInSession)
        );
    """
try:
    session.execute(query)
except Exception as e:
    print(e)              

In [85]:
# insert music app history data for the first query

df = data[['sessionId', 'itemInSession', 'artist', 'song', 'length']]

# create columns list for insertion

cols = ','.join([str(i) for i in df.columns.tolist()])

# insert DataFrame recrds one by one.

for (i, row) in df.iterrows():

    # create a query

    query = \
        "INSERT INTO artist_song_sessions ({}) \
        VALUES (%s, %s, %s, %s, %s)".format(cols)

    # insert record to table

    session.execute(query, tuple(row))

### Verification
    Do a SELECT to verify that the data have been inserted into each table

In [86]:
# Select the data for sessionId=338 and itemInSession=4, to verify that data insertion was correct

query = "SELECT artist, song, length FROM artist_song_sessions WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


---

###  Question 2:
> Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

---

***CREATE TABLE user_songs for the second query***

In [87]:
# Create a table artist_song_user_sessions for the second query with userId and sessionId as partition and 
# itemInSession as clustering key.
query = """
        CREATE TABLE  IF NOT EXISTS artist_song_user_sessions (
            userId INT,
            sessionId INT,
            itemInSession INT,
            artist TEXT,
            song TEXT,
            firstname TEXT,
            lastname TEXT,
            PRIMARY KEY ((userId, sessionId), itemInSession)
        );
    """
try:
    session.execute(query)
except Exception as e:
    print(e)                       

In [88]:
# insert music app history data for the second query

df = data[['userId', 'sessionId', 'itemInSession', 'artist', 'song', 'firstName', 'lastName']]

# create columns list for insertion

cols = ','.join([str(i) for i in df.columns.tolist()])

# insert DataFrame recrds one by one.

for (i, row) in df.iterrows():

    # create a query

    query = \
        "INSERT INTO artist_song_user_sessions ({}) \
        VALUES (%s, %s, %s, %s, %s, %s, %s)".format(cols)

    # insert record to table

    session.execute(query, tuple(row))

***Verification***
> Do a SELECT to verify that the data have been inserted into each table

In [89]:
# Select name of artist, song and user (first and last name) for userid = 10, sessionid = 182

query = \
    "SELECT artist, song, firstname, lastname FROM artist_song_user_sessions WHERE userId = 10 AND sessionId = 182"
try:
    results = session.execute(query)
except Exception as e:
    print(e)
    
for row in results:
    print(row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


---

###  Question 3
> Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

---

***CREATE TABLE user_songs for the third query***

In [90]:
query = """
        CREATE TABLE  IF NOT EXISTS user_songs (
            song TEXT,
            firstname TEXT,
            lastname TEXT,
            PRIMARY KEY (song)
        );
    """
try:
    session.execute(query)
except Exception as e:
    print(e)

In [91]:
# insert music app history data for the third query

df = data[['song','firstName', 'lastName']]

# create columns list for insertion

cols = ','.join([str(i) for i in df.columns.tolist()])

# insert DataFrame recrds one by one.

for (i, row) in df.iterrows():

    # create a query

    query = \
        "INSERT INTO user_songs ({}) \
        VALUES (%s, %s, %s)".format(cols)

    # insert record to table

    session.execute(query, tuple(row))

***Verification***
> Do a SELECT to verify that the data have been inserted into each table

In [92]:
# Select name of firstname, lastname and song for all users who listened to the song 'All Hands Against His Own'

query = "SELECT firstname, lastname FROM user_songs WHERE song='All Hands Against His Own'"
try:
    results = session.execute(query)
except Exception as e:
    print(e)
    
for row in results:
    print(row.firstname, row.lastname)

Tegan Levine


### Drop the tables before closing out the sessions

In [93]:
# Drop the table before closing out the sessions

tables = ['artist_song_sessions', 'artist_song_user_sessions', 'user_songs']

for table_name in tables:
    query = "drop table " + table_name
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [94]:
session.shutdown()
cluster.shutdown()